# Loading the Data

Loading the transformed data frame, previously saved on the Data Engineering stage

In [0]:
trainingData = spark.read.load("/FileStore/tables/trainingData")
testData = spark.read.load("/FileStore/tables/testData")

trainingData_2 = spark.read.load("/FileStore/tables/finalAIRBNB_noText")
testData_2 = spark.read.load("/FileStore/tables/finalAIRBNB_withText")


# Random Forest Regression Model

After some experimentation, the best set of hyperparameters was numTress=50 e maxDepth=20. After that, there wasn't much of improvement on the results.

In [0]:
from pyspark.ml.regression import RandomForestRegressor
import time 

tInitial = time.time()
rf = RandomForestRegressor(featuresCol="fts",labelCol='priceLog', numTrees=50, maxDepth=20)
rf_model = rf.fit(trainingData)
predictions_rf = rf_model.transform(testData)
finalTime = tInitial - time.time()

In [0]:
rf_model_2 = rf.fit(trainingData_2)
predictions_rf_2 = rf_model_2.transform(testData_2)

# Model Evaluation

As we can see, these are the results for the ensemble model. R-squared and RMSE are the metrics. 

* R-squared represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model. So, taking into consideration the log-transformed predicted variable, our model relies 54% on the features that we've given. This is reasonable result that can be improved later
* The RMSE metric isolated can't tell much to us, but in terms of applying an inverse log-transform, it might have a significant impact on the price target.

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="priceLog",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(predictions_rf))
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="priceLog",metricName="rmse")
print("RMSE on test data = %g" % lr_evaluator.evaluate(predictions_rf))

R Squared (R2) on test data = 0.538346
RMSE on test data = 0.18209

In [0]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="priceLog",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(predictions_rf_2))
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="priceLog",metricName="rmse")
print("RMSE on test data = %g" % lr_evaluator.evaluate(predictions_rf_2))

R Squared (R2) on test data = 0.547396
RMSE on test data = 0.181658

In [0]:
print("Time taken in RF Model: ", finalTime)

Time taken in RF Model: -209.67373251914978